# Example - URBANopt Project with REopt Gem 

In [1]:
require 'openstudio/extension'
require 'openstudio/extension/rake_task'
require 'urbanopt/scenario'
require 'urbanopt/geojson'
require 'urbanopt/reopt_scenario'
require 'urbanopt/reopt'
require 'daru'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [47]:
DEVELOPER_NREL_KEY = ''

<main>: warning: already initialized constant DEVELOPER_NREL_KEY


""

## Setup

In [18]:
def baseline_scenario
  root_dir = File.absolute_path(File.dirname(__FILE__))
  name = 'Baseline Scenario'
  run_dir = File.join(File.dirname(__FILE__), 'run/baseline_scenario/')
  feature_file_path = File.join(File.dirname(__FILE__), 'example_project.json')
  csv_file = File.join(File.dirname(__FILE__), 'baseline_scenario.csv')
  mapper_files_dir = File.join(File.dirname(__FILE__), '../mappers/')
  reopt_files_dir = File.join(File.dirname(__FILE__), '../reopt/')
  scenario_reopt_assumptions_file_name = 'base_assumptions.json'
  num_header_rows = 1

  feature_file = URBANopt::GeoJSON::GeoFile.from_file(feature_file_path)
  scenario = URBANopt::Scenario::REoptScenarioCSV.new(name, root_dir, run_dir, feature_file, mapper_files_dir, csv_file, num_header_rows, reopt_files_dir, scenario_reopt_assumptions_file_name)
  return scenario
end


:baseline_scenario

In [19]:
csv_file = File.join(File.dirname(__FILE__), 'baseline_scenario.csv')
Daru::DataFrame.from_csv csv_file

,Feature Id,Feature Name,Mapper Class,REopt Assumptions
0,1,Mixed_use 1,URBANopt::Scenario::BaselineMapper,base_assumptions.json
1,2,Restaurant 1,URBANopt::Scenario::BaselineMapper,base_assumptions.json
2,3,Residential 1,URBANopt::Scenario::BaselineMapper,base_assumptions.json


In [20]:
feature_reports_hash = nil
File.open('../reopt/base_assumptions.json', 'r') do |file|
  feature_reports_hash = JSON.parse(file.read)
end

p "PV", feature_reports_hash["Scenario"]['Site']['PV']
p "Storage", feature_reports_hash["Scenario"]['Site']['Storage']
p "ElectricTariff", feature_reports_hash["Scenario"]['Site']['ElectricTariff']
p "Financial", feature_reports_hash["Scenario"]['Site']['Financial']


"PV"
{"existing_kw"=>0, "min_kw"=>0, "max_kw"=>1000000000.0, "installed_cost_us_dollars_per_kw"=>1600, "om_cost_us_dollars_per_kw"=>16, "macrs_option_years"=>5, "macrs_bonus_pct"=>1, "macrs_itc_reduction"=>0.5, "federal_itc_pct"=>0.26, "state_ibi_pct"=>0, "state_ibi_max_us_dollars"=>10000000000.0, "utility_ibi_pct"=>0, "utility_ibi_max_us_dollars"=>10000000000.0, "federal_rebate_us_dollars_per_kw"=>0, "state_rebate_us_dollars_per_kw"=>0, "state_rebate_max_us_dollars"=>10000000000.0, "utility_rebate_us_dollars_per_kw"=>0, "utility_rebate_max_us_dollars"=>10000000000.0, "pbi_us_dollars_per_kwh"=>0, "pbi_max_us_dollars"=>1000000000.0, "pbi_years"=>1, "pbi_system_max_kw"=>1000000000.0, "degradation_pct"=>0.005, "azimuth"=>180, "losses"=>0.14, "array_type"=>1, "module_type"=>0, "gcr"=>0.4, "dc_ac_ratio"=>1.2, "inv_eff"=>0.96, "radius"=>0, "tilt"=>0.537}
"Storage"
{"min_kw"=>0, "max_kw"=>1000000, "min_kwh"=>0, "max_kwh"=>1000000, "internal_efficiency_pct"=>0.975, "inverter_efficiency_pct"=>0

["Financial", {"om_cost_escalation_pct"=>0.025, "escalation_pct"=>0.023, "offtaker_tax_pct"=>0.26, "offtaker_discount_pct"=>0.083, "analysis_years"=>25, "value_of_lost_load_us_dollars_per_kwh"=>100, "microgrid_upgrade_cost_pct"=>0.3}]

## Post Processing

In [21]:
default_post_processor = URBANopt::Scenario::ScenarioDefaultPostProcessor.new(baseline_scenario)
scenario_report = default_post_processor.run
scenario_base = default_post_processor.scenario_base
scenario_report.save('baseline_scenario')

true

In [22]:
reopt_post_processor = URBANopt::REopt::REoptPostProcessor.new(scenario_report,scenario_base.scenario_reopt_assumptions_file, scenario_base.reopt_feature_assumptions, DEVELOPER_NREL_KEY) 
true

true

In [23]:
p "Scenario Level Assumptions", reopt_post_processor.scenario_reopt_default_assumptions_hash
p
p "Feature Level Assumptions", reopt_post_processor.feature_reports_reopt_default_assumption_hashes

"Scenario Level Assumptions"
{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebate_

[{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebate_max_us_dollars=>10000000000.

["Feature Level Assumptions", [{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebat

In [24]:
p 'Scenario REopt Response',reopt_post_processor.scenario_reopt_default_output_file
p 'New Scenario Timeseries CSV', reopt_post_processor.scenario_timeseries_default_output_file
p 'Feature REopt Responses',reopt_post_processor.feature_reports_reopt_default_output_files
p 'New Feature Timeseries CSVs', reopt_post_processor.feature_reports_timeseries_default_output_files

"Scenario REopt Response"
"./run/baseline_scenario/scenario_report_Baseline Scenario_reopt_run.json"
"New Scenario Timeseries CSV"
"./run/baseline_scenario/scenario_report_Baseline Scenario_timeseries.csv"
"Feature REopt Responses"
["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_reopt_run.json", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_reopt_run.json", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_reopt_run.json"]
"New Feature Timeseries CSVs"
["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_timeseries.csv"]


["New Feature Timeseries CSVs", ["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_timeseries.csv"]]

In [26]:
scenario_report_reopt = URBANopt::Scenario::DefaultReports::ScenarioReport.new(scenario_report.to_hash)
scenario_report_reopt = reopt_post_processor.run_scenario_report(scenario_report_reopt)
scenario_report_reopt.save('baseline_scenario_reopt')

true

In [27]:
reopt_post_processor.scenario_reopt_default_assumptions_hash = nil
scenario_report_default = URBANopt::Scenario::DefaultReports::ScenarioReport.new(scenario_report.to_hash)
scenario_report_scenario_default = reopt_post_processor.run_scenario_report(scenario_report_default,nil,
  "./run/baseline_scenario/scenario_report_Baseline Scenario_reopt_run_DEFAULT.json",
  "./run/baseline_scenario/scenario_report_Baseline Scenario_timeseries_DEFAULT.csv")
scenario_report_scenario_default.save('baseline_scenario_DEFAULT')

true

In [40]:
scenario_report_reoptOnFeatures = reopt_post_processor.run_scenario_report_features(scenario_report)
scenario_report_reoptOnFeatures.save('baseline_scenario_REoptAtEachFeature')

true

## Results

In [ ]:
s_pv = scenario_report_reopt.distributed_generation.total_solar_pv_kw
s_storage_kw = scenario_report_reopt.distributed_generation.total_storage_kw
s_storage_kwh = scenario_report_reopt.distributed_generation.total_storage_kwh
s_lcc = scenario_report_reopt.distributed_generation.lcc_us_dollars
s_savings = scenario_report_reopt.distributed_generation.npv_us_dollars/scenario_report_scenario.distributed_generation.lcc_us_dollars

f_pv = scenario_report_reoptOnFeatures.distributed_generation.total_solar_pv_kw
f_storage_kw = scenario_report_reoptOnFeatures.distributed_generation.total_storage_kw
f_storage_kwh = scenario_report_reoptOnFeatures.distributed_generation.total_storage_kwh
f_lcc = scenario_report_reoptOnFeatures.distributed_generation.lcc_us_dollars
f_pct_savings = scenario_report_reoptOnFeatures.distributed_generation.npv_us_dollars/scenario_report_features.distributed_generation.lcc_us_dollars

TypeError: nil can't be coerced into Float

In [42]:
#PV Capacity
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_pv,f_pv]})
p1 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kW"
end

#<Nyaplot::Plot:0x007f853b718910 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f8522032510 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"9d755176-cc51-4b10-917a-5ab4515e4fbf"}, @xrange=["scenario", "feature"], @yrange=[0, 10041.575]>], :options=>{:x_label=>"", :y_label=>"kW", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 10041.575]}}>

In [43]:
#Storage kW
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_storage_kw,f_storage_kw]})
p2 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kW"
end

#<Nyaplot::Plot:0x007f852150fa70 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f852150ca28 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"73633f6c-e7a9-4f0c-bb1a-b243021988f5"}, @xrange=["scenario", "feature"], @yrange=[0, 733.7625]>], :options=>{:x_label=>"", :y_label=>"kW", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 733.7625]}}>

In [44]:
#Storage kWh
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_storage_kwh,f_storage_kwh]})
p3 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kWh"
end

#<Nyaplot::Plot:0x007f853c449c60 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f85406af668 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"9aaa88d0-62be-4025-85c0-8a7f8d71fb69"}, @xrange=["scenario", "feature"], @yrange=[0, 1512.3453]>], :options=>{:x_label=>"", :y_label=>"kWh", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 1512.3453]}}>

In [45]:
#Lifecycle Costs
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_lcc,f_lcc]})
p4 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "$"
end

#<Nyaplot::Plot:0x007f853f8d91b0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f854062ad00 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"9e263ca9-9165-481a-9fc6-a2e8fccbbb35"}, @xrange=["scenario", "feature"], @yrange=[0, 44955059.0]>], :options=>{:x_label=>"", :y_label=>"$", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 44955059.0]}}>

In [46]:
#Percent Savings
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_savings,f_pct_savings]})
p5 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "%"
end

#<Nyaplot::Plot:0x007f8521f34bb8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f8521906868 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"362df447-c82a-4636-8f38-892b7910ad77"}, @xrange=["scenario", "feature"], @yrange=[0, 0.2195232045711258]>], :options=>{:x_label=>"", :y_label=>"%", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 0.2195232045711258]}}>